In [1]:
from __future__ import print_function

In [2]:
import numpy as np
import axidma
reload(axidma)


def to_bin(x):
    vv = [ (x>>(32-(i+1)*4))&0xF for i in range(8)]
    return '{0:04b}_{1:04b}|{2:04b}_{3:04b}|{4:04b}_{5:04b}|{6:04b}_{7:04b}'.format( *vv )

def dump_offsets(dt):
    return '\n'.join(sorted(['{:02X}h {}'.format((v[1]),k) for (k,v) in dt.fields.iteritems() ]))

dma = axidma.AxiDMA_SG('dma')

axidma.dma_is_sg(dma.s2mm), axidma.axi_dma_direct_dtype.itemsize, axidma.axi_dma_sg_dtype.itemsize

(True, 48, 48)

In [30]:
print('\n'.join([ to_bin(dma.mm2s[k]) + " -- "+k for k in dma.mm2s.dtype.fields.keys() if k != 'reserved']))

0000_0000|0000_0000|0000_0000|0000_0000 -- curr_desc
0000_0000|0000_0001|0000_0000|0000_1001 -- st
0000_0000|0000_0000|0000_0000|0000_0000 -- tail_desc_msb
0000_0000|0000_0001|0000_0000|0000_0010 -- cr
0000_0000|0000_0000|0000_0000|0000_0011 -- sg_ctl
0000_0000|0000_0000|0000_0000|0000_0000 -- curr_desc_msb
0000_0000|0000_0000|0000_0000|0000_0000 -- tail_desc


In [68]:
def get_bit(v, bit):
    return 0 if (v&(1<<bit)) == 0 else 1

def get_bits(v, msb, lsb):
    assert msb >= lsb
    nb   = msb-lsb+1
    mask = (1<<nb)-1
    return (v>>lsb)&mask

def set_bits(v, msb, lsb, x_bits):
    assert msb >= lsb
    nb   = msb-lsb+1
    mask = ((1<<nb)-1)<<lsb
    
    assert x_bits < (1<<nb)
    
    return (v&(~mask))|(x_bits << lsb)


    
x = dma.s2mm.cr

#[(i,get_bit(x,i)) for i in range(32)]
hex(set_bits(0xF3, 7,7, 0))

'0x73'

In [43]:
sg_desc_dtype = np.dtype([
        ('next'               , '<u4'),
        ('next_msb'           , '<u4'),
        ('buffer_address'     , '<u4'),
        ('buffer_address_msb' , '<u4'),
        ('reserved'           , '<u4',2),
        ('control'            , '<u4'),
        ('status'             , '<u4'),
        ('app'                , '<u4',5),
        ('padding'            , '<u4',3)])


print(dump_offsets(sg_desc_dtype))

00h next
04h next_msb
08h buffer_address
0Ch buffer_address_msb
10h reserved
18h control
1Ch status
20h app
34h padding


In [46]:
axi_dma_sg_dtype = np.dtype([
        ('cr'             , '<u4'),
        ('st'             , '<u4'),
        ('curr_desc'      , '<u4'),
        ('curr_desc_msb'  , '<u4'),
        ('tail_desc'      , '<u4'),
        ('tail_desc_msb'  , '<u4'),
        ('reserved'       , '<u4',5),
        ('sg_ctl'         , '<u4'),])


print(dump_offsets(axi_dma_sg_dtype))
axi_dma_sg_dtype.itemsize, 0x30, hex(48), (0x2C-0x14-4)//4

00h cr
04h st
08h curr_desc
0Ch curr_desc_msb
10h tail_desc
14h tail_desc_msb
18h reserved
2Ch sg_ctl


(48, 48, '0x30', 5)

In [3]:
print('\n'.join([to_bin(x) for x in [dma.mm2s.cr, dma.s2mm.cr, dma.mm2s.st,dma.s2mm.st]]))

0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0001|0000_0000|0000_1001
0000_0000|0000_0001|0000_0000|0000_1001


In [4]:
import uio
reload(uio)
mem_uio = uio.UIO('scratch_mem')

SZ=(1<<19)

data = mem_uio.as_ndarray()

src_data = data[:SZ]
dst_data = data[SZ:SZ*2]

src_data[:] = np.random.randint(0,256,SZ)
dst_data[:] = 0xFF

print("SRC:",src_data[:4])
print("DST:",dst_data[:4])

SRC: [ 19 105   7 192]
DST: [255 255 255 255]


In [5]:
dma.launch(mem_uio.phy_buf(src_data), mem_uio.phy_buf(dst_data), enable_interrupts=True)
if dma.wait():
    print('DMA Completed Succesfully')
else:
    print('DMA FAILED')



DMA Completed Succesfully


In [6]:
if (src_data == dst_data).all():
    print('SUCCESS: Data copied as expected')
else:
    print('FAILED: dst and src do not match up')

print("SRC:",src_data[:4],'...')
print("DST:",dst_data[:4],'...')

SUCCESS: Data copied as expected
SRC: [ 19 105   7 192] ...
DST: [ 19 105   7 192] ...


In [3]:
%%sh

date
whoami
ls -l /dev/uio* /dev/mem
cat /sys/class/uio/uio?/maps/map0/name
cat /sys/class/uio/uio?/name
ls -l /sys/class/uio/uio0/maps/map0

Wed Mar 16 11:44:16 UTC 2016
parallella
crw-r----- 1 root kmem      1, 1 Jan  1  1970 /dev/mem
crw-rw---- 1 root plugdev 247, 0 Jan  1  1970 /dev/uio0
crw-rw---- 1 root plugdev 247, 1 Jan  1  1970 /dev/uio1
crw-rw---- 1 root plugdev 247, 2 Jan  1  1970 /dev/uio2
/scratch_mem@3e000000
/amba_pl/my_mult@70020000
/amba_pl/dma@40400000
scratch_mem
my_mult
dma
total 0
-r--r--r-- 1 root root 4096 Feb 24 12:03 addr
-r--r--r-- 1 root root 4096 Mar 16 11:44 name
-r--r--r-- 1 root root 4096 Mar 16 11:44 offset
-r--r--r-- 1 root root 4096 Feb 24 12:03 size


In [ ]:
#!dtc -I fs -O dts /proc/device-tree/ 